In [1]:
# Import Libs

from bs4 import BeautifulSoup
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes
from geopy.geocoders import Nominatim
import folium
from pandas.io.json import json_normalize
import numpy as np
from sklearn.cluster import KMeans
import geopy
import matplotlib.cm as cm
import matplotlib.colors as colors
import math
import urllib.request
import json
print('Installed Libraries')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Installed Libraries


Get Data

In [2]:
url = 'https://cocl.us/new_york_dataset'
filename = 'newyork_data.json'
urllib.request.urlretrieve(url, filename)
#!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
scraped_data = BeautifulSoup(requests.get(url).text, 'lxml')
print('Data downloaded!')
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
print('New York Data' , )
print(newyork_data)
print('\n')
print('Toronto Data ' , )
print(scraped_data)
print('\n')

Data downloaded!
New York Data
{'type': 'FeatureCollection', 'totalFeatures': 306, 'features': [{'type': 'Feature', 'id': 'nyu_2451_34572.1', 'geometry': {'type': 'Point', 'coordinates': [-73.84720052054902, 40.89470517661]}, 'geometry_name': 'geom', 'properties': {'name': 'Wakefield', 'stacked': 1, 'annoline1': 'Wakefield', 'annoline2': None, 'annoline3': None, 'annoangle': 0.0, 'borough': 'Bronx', 'bbox': [-73.84720052054902, 40.89470517661, -73.84720052054902, 40.89470517661]}}, {'type': 'Feature', 'id': 'nyu_2451_34572.2', 'geometry': {'type': 'Point', 'coordinates': [-73.82993910812398, 40.87429419303012]}, 'geometry_name': 'geom', 'properties': {'name': 'Co-op City', 'stacked': 2, 'annoline1': 'Co-op', 'annoline2': 'City', 'annoline3': None, 'annoangle': 0.0, 'borough': 'Bronx', 'bbox': [-73.82993910812398, 40.87429419303012, -73.82993910812398, 40.87429419303012]}}, {'type': 'Feature', 'id': 'nyu_2451_34572.3', 'geometry': {'type': 'Point', 'coordinates': [-73.82780644716412, 40

Transform into Dataframe

In [3]:
newyork_neighbourhoods_data = newyork_data['features']
newyork_column_names = ['Borough', 'Neighbourhood', 'Latitude', 'Longitude'] 
newyork_neighbourhoods = pd.DataFrame(columns=newyork_column_names)
for data in newyork_neighbourhoods_data:
    borough = neighbourhood_name = data['properties']['borough'] 
    neighbourhood_name = data['properties']['name']
        
    neighbourhood_latlon = data['geometry']['coordinates']
    neighbourhood_lat = neighbourhood_latlon[1]
    neighbourhood_lon = neighbourhood_latlon[0]
    
    newyork_neighbourhoods = newyork_neighbourhoods.append({'Borough': borough,
                                          'Neighbourhood': neighbourhood_name,
                                          'Latitude': neighbourhood_lat,
                                          'Longitude': neighbourhood_lon}, ignore_index=True)

toronto_cols = ['PostalCode', 'Borough', 'Neighbourhood']
toronto_neighbour_df = pd.DataFrame(columns=toronto_cols)

j = 0
for i in scraped_data.find('table').find_all('tr'):
    if (len(i.find_all('td')) > 0):
        PC = i.find_all('td')[0].text.strip()
        B = i.find_all('td')[1].text.strip()
        N = i.find_all('td')[2].text.strip()
        toronto_neighbour_df.loc[j] = [i.find_all('td')[0].text.strip(), i.find_all('td')[1].text.strip(), i.find_all('td')[2].text.strip()]
        j = j + 1

Clean Data

In [4]:
toronto_neighbour_df = toronto_neighbour_df[toronto_neighbour_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_neighbour_df = toronto_neighbour_df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
toronto_neighbour_df.columns = ['PostalCode', 'Borough', 'Neighbourhood']
for i, j in toronto_neighbour_df.iterrows():
    if j['Neighbourhood'] == 'Not assigned':
        j['Neighbourhood'] = j['Borough']
print(toronto_neighbour_df)
toronto_neighbour_df.shape

    PostalCode           Borough  \
0          M1B       Scarborough   
1          M1C       Scarborough   
2          M1E       Scarborough   
3          M1G       Scarborough   
4          M1H       Scarborough   
5          M1J       Scarborough   
6          M1K       Scarborough   
7          M1L       Scarborough   
8          M1M       Scarborough   
9          M1N       Scarborough   
10         M1P       Scarborough   
11         M1R       Scarborough   
12         M1S       Scarborough   
13         M1T       Scarborough   
14         M1V       Scarborough   
15         M1W       Scarborough   
16         M1X       Scarborough   
17         M2H        North York   
18         M2J        North York   
19         M2K        North York   
20         M2L        North York   
21         M2M        North York   
22         M2N        North York   
23         M2P        North York   
24         M2R        North York   
25         M3A        North York   
26         M3B        North 

(103, 3)

Maps

In [5]:
newyork_address = 'New York City, NY'

geolocator = Nominatim(user_agent='my_application')
newyork_location = geolocator.geocode(newyork_address)
newyork_latitude = newyork_location.latitude
newyork_longitude = newyork_location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(newyork_latitude, newyork_longitude))

map_newyork = folium.Map(location=[newyork_latitude, newyork_longitude], zoom_start=10)

for lat, lng, borough, neighbourhood in zip(newyork_neighbourhoods['Latitude'], newyork_neighbourhoods['Longitude'], newyork_neighbourhoods['Borough'], newyork_neighbourhoods['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [6]:
toronto_gd = pd.read_csv('http://cocl.us/Geospatial_data')
toronto_gd.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
toronto_geo_neighbour_df = pd.merge(toronto_neighbour_df, toronto_gd, on="PostalCode", how='left')
toronto_geo_neighbour_df

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [7]:
geopy.geocoders.options.default_timeout = None

toronto_address = 'Toronto, Canada'

geolocator = Nominatim()
toronto_location = geolocator.geocode(toronto_address)
toronto_latitude = toronto_location.latitude
toronto_longitude = toronto_location.longitude
toronto_map = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=10)

for lat, lng, borough, neighbourhood in zip(toronto_geo_neighbour_df['Latitude'], toronto_geo_neighbour_df['Longitude'], toronto_geo_neighbour_df['Borough'], toronto_geo_neighbour_df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#1aff66',
        fill_opacity=0.5,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

D:\Softwares\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """


In [8]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
CLIENT_ID = 'BRXRTBE1IEXJYVUED44XMZK51AWXU2MYQD1YCCTRXS2T0OHS' # your Foursquare ID
CLIENT_SECRET = 'GTBNTSB1ZOBMGS4U1Q1EHZSLZLBHOQNYVTEMMSRG50PZ1ODB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BRXRTBE1IEXJYVUED44XMZK51AWXU2MYQD1YCCTRXS2T0OHS
CLIENT_SECRET:GTBNTSB1ZOBMGS4U1Q1EHZSLZLBHOQNYVTEMMSRG50PZ1ODB


In [11]:
LIMIT = 100
radius = 500

In [12]:
newyork_venues = getNearbyVenues(names=newyork_neighbourhoods['Neighbourhood'],
                                   latitudes=newyork_neighbourhoods['Latitude'],
                                   longitudes=newyork_neighbourhoods['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [13]:
toronto_venues = getNearbyVenues(names=toronto_geo_neighbour_df['Neighbourhood'],
                                   latitudes=toronto_geo_neighbour_df['Latitude'],
                                   longitudes=toronto_geo_neighbour_df['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [14]:
print('New York Venue Shape' , newyork_venues.shape)
#newyork_venues.head()
print('Toronto Venue Shape', toronto_venues.shape)
#toronto_venues.head()

New York Venue Shape (9863, 7)
Toronto Venue Shape (2112, 7)


In [15]:
#print('Venues returned by New York Neighbourhoods : \n')
#print(newyork_venues.groupby('Neighbourhood').count())
newyork_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Allerton,26,26,26,26,26,26
Annadale,12,12,12,12,12,12
Arden Heights,6,6,6,6,6,6
Arlington,7,7,7,7,7,7
Arrochar,21,21,21,21,21,21
Arverne,18,18,18,18,18,18
Astoria,100,100,100,100,100,100
Astoria Heights,16,16,16,16,16,16
Auburndale,17,17,17,17,17,17


In [16]:
#print('Venues returned by Toronto Neighbourhoods : \n')
#print(toronto_venues.groupby('Neighbourhood').count())
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Wilson Heights, Downsview North",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,56,56,56,56,56,56
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17


In [17]:
print('There are {} uniques categories for New York.'.format(len(newyork_venues['Venue Category'].unique())))

There are 430 uniques categories for New York.


In [18]:
print('There are {} uniques categories for Toronto.'.format(len(toronto_venues['Venue Category'].unique())))

There are 272 uniques categories for Toronto.


Analyzing Each Neighbourhood

In [19]:
newyork_onehot = pd.get_dummies(newyork_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

newyork_onehot['Neighbourhood'] = newyork_venues['Neighbourhood'] 
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

newyork_fixed_columns = [newyork_onehot.columns[-1]] + list(newyork_onehot.columns[:-1])
newyork_onehot = newyork_onehot[newyork_fixed_columns]
toronto_fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[toronto_fixed_columns]

newyork_grouped = newyork_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()


Let's Compare most common venuew for both places

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

New York

In [21]:
num_top_venues = 5

for hood in newyork_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = newyork_grouped[newyork_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
            venue  freq
0     Pizza Place  0.15
1   Deli / Bodega  0.08
2     Supermarket  0.08
3  Discount Store  0.04
4  Breakfast Spot  0.04


----Annadale----
                 venue  freq
0          Pizza Place  0.25
1               Bakery  0.08
2        Deli / Bodega  0.08
3  American Restaurant  0.08
4           Restaurant  0.08


----Arden Heights----
         venue  freq
0     Pharmacy  0.17
1         Pool  0.17
2  Coffee Shop  0.17
3  Pizza Place  0.17
4   Playground  0.17


----Arlington----
           venue  freq
0       Bus Stop  0.29
1  Deli / Bodega  0.14
2    Coffee Shop  0.14
3   Home Service  0.14
4           Tree  0.14


----Arrochar----
                venue  freq
0       Deli / Bodega  0.10
1         Pizza Place  0.10
2  Italian Restaurant  0.10
3            Bus Stop  0.10
4        Liquor Store  0.05


----Arverne----
            venue  freq
0       Surf Spot  0.22
1   Metro Station  0.11
2  Sandwich Place  0.11
3     Coffee Shop  0.06
4     Pizza P

4           Grocery Store  0.07


----City Line----
                       venue  freq
0                 Donut Shop  0.09
1                 Shoe Store  0.06
2        Fried Chicken Joint  0.06
3              Grocery Store  0.06
4  South American Restaurant  0.03


----Civic Center----
                 venue  freq
0          Coffee Shop  0.07
1    French Restaurant  0.05
2                 Park  0.04
3  American Restaurant  0.04
4                Hotel  0.04


----Claremont Village----
           venue  freq
0  Grocery Store  0.17
1    Bus Station  0.17
2    Pizza Place  0.11
3           Food  0.06
4         Bakery  0.06


----Clason Point----
                       venue  freq
0                       Park   0.4
1                       Pool   0.1
2        American Restaurant   0.1
3                   Bus Stop   0.1
4  South American Restaurant   0.1


----Clifton----
           venue  freq
0  Grocery Store  0.10
1  Train Station  0.10
2       Bus Stop  0.05
3           Park  0.05
4        

                       venue  freq
0                Pizza Place  0.17
1                   Pharmacy  0.11
2                Bus Station  0.11
3  Latin American Restaurant  0.11
4                        Gym  0.06


----Highland Park----
                venue  freq
0        Liquor Store  0.13
1       Grocery Store  0.13
2  Spanish Restaurant  0.07
3                Park  0.07
4       Big Box Store  0.07


----Hillcrest----
                       venue  freq
0                 Donut Shop  0.11
1  College Academic Building  0.05
2                 Food Court  0.05
3            College Stadium  0.05
4             Ice Cream Shop  0.05


----Hollis----
                 venue  freq
0       Baseball Field  0.13
1  Fried Chicken Joint  0.13
2        Shopping Mall  0.13
3                 Park  0.13
4       Discount Store  0.07


----Holliswood----
               venue  freq
0        Supermarket   0.2
1         Donut Shop   0.2
2  Mobile Phone Shop   0.2
3         Playground   0.2
4     Sandwich Place 

                venue  freq
0               Beach  0.17
1    Basketball Court  0.08
2  Chinese Restaurant  0.08
3       Deli / Bodega  0.08
4        Dessert Shop  0.08


----Midtown----
         venue  freq
0  Coffee Shop  0.07
1        Hotel  0.04
2      Theater  0.04
3       Bakery  0.03
4  Pizza Place  0.03


----Midtown South----
                 venue  freq
0    Korean Restaurant  0.12
1                Hotel  0.07
2  Japanese Restaurant  0.04
3         Burger Joint  0.04
4                 Café  0.04


----Midwood----
               venue  freq
0        Pizza Place  0.36
1         Bagel Shop  0.09
2           Pharmacy  0.09
3     Ice Cream Shop  0.09
4  Convenience Store  0.09


----Mill Basin----
                 venue  freq
0          Pizza Place  0.09
1  Japanese Restaurant  0.09
2     Sushi Restaurant  0.06
3   Chinese Restaurant  0.06
4                 Bank  0.06


----Mill Island----
                   venue  freq
0                   Pool   1.0
1  Outdoors & Recreation   0.0


4    Martial Arts Dojo  0.17


----Prince's Bay----
                venue  freq
0         Pizza Place  0.22
1           Pet Store  0.11
2            Pharmacy  0.11
3    Sushi Restaurant  0.11
4  Italian Restaurant  0.11


----Prospect Heights----
                venue  freq
0                 Bar  0.09
1  Mexican Restaurant  0.07
2     Thai Restaurant  0.04
3                Café  0.04
4        Gourmet Shop  0.04


----Prospect Lefferts Gardens----
                  venue  freq
0                  Café  0.08
1                Bakery  0.08
2  Caribbean Restaurant  0.06
3           Pizza Place  0.06
4    Spanish Restaurant  0.04


----Prospect Park South----
                  venue  freq
0  Caribbean Restaurant  0.09
1         Grocery Store  0.06
2  Fast Food Restaurant  0.06
3     Mobile Phone Shop  0.06
4           Pizza Place  0.06


----Queens Village----
               venue  freq
0               Bank  0.14
1           Bus Stop  0.14
2         Donut Shop  0.07
3  Martial Arts Dojo  0.07

4   Liquor Store  0.04


----Woodlawn----
                 venue  freq
0        Deli / Bodega  0.13
1          Pizza Place  0.13
2                  Pub  0.13
3           Playground  0.09
4  Rental Car Location  0.04


----Woodrow----
           venue  freq
0       Pharmacy  0.10
1    Coffee Shop  0.05
2         Bakery  0.05
3           Bank  0.05
4  Grocery Store  0.05


----Woodside----
                       venue  freq
0              Grocery Store  0.08
1            Thai Restaurant  0.06
2                        Bar  0.05
3  Latin American Restaurant  0.05
4        Filipino Restaurant  0.05


----Yorkville----
                venue  freq
0         Coffee Shop  0.08
1  Italian Restaurant  0.07
2                 Gym  0.06
3    Sushi Restaurant  0.04
4                 Bar  0.04




Toronto

In [22]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0               Skating Rink  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3  Latin American Restaurant  0.25
4              Metro Station  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place   0.2
1        Pharmacy   0.1
2             Pub   0.1
3  Sandwich Place   0.1
4             Gym   0.1


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                Coffee Shop  0.11
1                       Bank  0.11
2                Pizza Place  0.05
3              Shopping Mall  0.05
4  Middle Eastern Restaurant  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Bank  0.25
2   Chinese Restaurant  0.25
3                 Café  0.25
4  Moroccan Restaurant  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.09
1      Sandwich Place  0.09

               venue  freq
0  Convenience Store   0.2
1   Department Store   0.2
2        Bus Station   0.2
3        Coffee Shop   0.2
4     Discount Store   0.2


----Kensington Market, Chinatown, Grange Park----
                   venue  freq
0                   Café  0.09
1     Mexican Restaurant  0.05
2  Vietnamese Restaurant  0.05
3                 Bakery  0.05
4            Coffee Shop  0.05


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
               venue  freq
0               Park  0.25
1        Pizza Place  0.25
2     Sandwich Place  0.25
3  Mobile Phone Shop  0.25
4      Metro Station  0.00


----Lawrence Manor, Lawrence Heights----
                 venue  freq
0    Accessories Store  0.09
1  Arts & Crafts Store  0.09
2          Coffee Shop  0.09
3       Clothing Store  0.09
4         Carpet Store  0.09


----Lawrence Park----
                 venue  freq
0                 Park  0.25
1   Dim Sum Restaurant  0.25
2          Swim School  0.25

                    venue  freq
0        Greek Restaurant  0.21
1      Italian Restaurant  0.07
2             Coffee Shop  0.07
3  Furniture / Home Store  0.05
4          Ice Cream Shop  0.05


----The Kingsway, Montgomery Road, Old Mill North----
                             venue  freq
0                             Park  0.33
1                            River  0.33
2                             Pool  0.33
3               Mexican Restaurant  0.00
4  Molecular Gastronomy Restaurant  0.00


----Thorncliffe Park----
               venue  freq
0  Indian Restaurant  0.10
1        Yoga Studio  0.05
2     Discount Store  0.05
3       Burger Joint  0.05
4     Sandwich Place  0.05


----Toronto Dominion Centre, Design Exchange----
                 venue  freq
0          Coffee Shop  0.10
1                Hotel  0.06
2                 Café  0.06
3           Restaurant  0.05
4  Japanese Restaurant  0.03


----University of Toronto, Harbord----
                venue  freq
0                Café  

Top Venue for each neighbourhood in New York

In [23]:
num_top_venues = 1

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

newyork_neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
newyork_neighbourhoods_venues_sorted['Neighbourhood'] = newyork_grouped['Neighbourhood']

for ind in np.arange(newyork_grouped.shape[0]):
    newyork_neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(newyork_grouped.iloc[ind, :], num_top_venues)

newyork_neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue
0,Allerton,Pizza Place
1,Annadale,Pizza Place
2,Arden Heights,Playground
3,Arlington,Bus Stop
4,Arrochar,Deli / Bodega
5,Arverne,Surf Spot
6,Astoria,Middle Eastern Restaurant
7,Astoria Heights,Italian Restaurant
8,Auburndale,Hookah Bar
9,Bath Beach,Chinese Restaurant


Top Venue for each Neighbourhood in Toronto

In [24]:
num_top_venues = 1

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

toronto_neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
toronto_neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue
0,Agincourt,Lounge
1,"Alderwood, Long Branch",Pizza Place
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank
3,Bayview Village,Café
4,"Bedford Park, Lawrence Manor East",Sushi Restaurant
5,Berczy Park,Coffee Shop
6,"Birch Cliff, Cliffside West",College Stadium
7,"Brockton, Parkdale Village, Exhibition Place",Café
8,"Business reply mail Processing Centre, South C...",Yoga Studio
9,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service


Clustering both places

Clustering New York

In [43]:
# set number of clusters
kclusters = 5

newyork_grouped_clustering = newyork_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(newyork_grouped_clustering)

# add clustering labels
newyork_neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

newyork_merged = newyork_neighbourhoods

newyork_merged = newyork_merged.join(newyork_neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

# create map
newyork_map_clusters = folium.Map(location=[newyork_latitude, newyork_longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(newyork_merged['Latitude'], newyork_merged['Longitude'], newyork_merged['Neighbourhood'], newyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if (math.isnan(cluster)):
        cluster = 1
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(newyork_map_clusters)
       
newyork_map_clusters

Clustering Toronto

In [42]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# add clustering labels
toronto_neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_geo_neighbour_df

toronto_merged = toronto_merged.join(toronto_neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

# create map
toronto_map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if (math.isnan(cluster)):
        cluster = 1
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(toronto_map_clusters)
       
toronto_map_clusters

Let's compare clusters

1st Cluster

In [30]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 0, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue
3,Fieldston,Plaza
4,Riverdale,Park
9,Williamsbridge,Caribbean Restaurant
12,City Island,Harbor / Marina
22,Port Morris,Latin American Restaurant
27,Clason Point,Park
28,Throgs Neck,Italian Restaurant
35,Spuyten Duyvil,Intersection
37,Pelham Bay,Italian Restaurant
39,Edgewater Park,Italian Restaurant


In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue
5,Scarborough,0.0,Playground
14,Scarborough,0.0,Park
23,North York,0.0,Park
40,East York,0.0,Park
50,Downtown Toronto,0.0,Park
98,York,0.0,Park


2nd Cluster

In [32]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 1, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue
192,Somerville,Park
203,Todt Hill,Park
303,Bayswater,Park


In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue
1,Scarborough,1.0,Construction & Landscaping
2,Scarborough,1.0,Medical Center
3,Scarborough,1.0,Coffee Shop
4,Scarborough,1.0,Hakka Restaurant
6,Scarborough,1.0,Department Store
7,Scarborough,1.0,Bakery
8,Scarborough,1.0,American Restaurant
9,Scarborough,1.0,College Stadium
10,Scarborough,1.0,Indian Restaurant
11,Scarborough,1.0,Auto Garage


3rd Cluster

In [34]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 2, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue
0,Wakefield,Pharmacy
1,Co-op City,Bus Station
2,Eastchester,Caribbean Restaurant
5,Kingsbridge,Pizza Place
6,Marble Hill,Sandwich Place
7,Woodlawn,Deli / Bodega
8,Norwood,Pizza Place
10,Baychester,Bank
11,Pelham Parkway,Italian Restaurant
13,Bedford Park,Diner


In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue
96,North York,2.0,Shopping Mall


4th Cluster

In [36]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 3, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue
198,New Brighton,Bus Stop
202,Grymes Hill,Dog Run
224,Park Hill,Bus Stop
226,Graniteville,Bus Stop
227,Arlington,Bus Stop
256,Randall Manor,Bus Stop
258,Elm Park,Bus Stop
285,Willowbrook,Bus Stop
286,Sandy Ground,Bus Stop
305,Fox Hills,Bus Station


In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue
0,Scarborough,3.0,Fast Food Restaurant


5th Cluster

In [38]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 4, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue
207,Port Ivory,Business Service


In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue
94,Etobicoke,4.0,Golf Course
